In [ ]:
%pip install python-terrier 
%pip install --upgrade python-terrier


In [1]:
import pyterrier as pt
import pandas as pd

if not pt.started():
  pt.init()


/home/christian/Desktop/information_retrieval/scrapy_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
df = pd.read_json("./space/results_esa.json")
ids = []
for i in range(len(df)):
    ids.append(f"d{i}")
df['docno'] = ids
df['text'] = df['mission_name'] + df['mission_description'] #+ df['mission_page']
# df.drop(['mission_name','mission_description','mission_page'],axis=1,inplace=True)
# # df.head(5)
df

,mission_name,mission_description,mission_page,docno,text
0,Aeolus,Aeolus was the first satellite mission to acqu...,https://www.esa.int/Applications/Observing_the...,d0,Aeolus Aeolus was the first satellite mission ...
1,Φ-sat,"Ф-sat-1, an enhancement of the Federated Satel...",https://www.esa.int/Applications/Observing_the...,d1,"Φ-sat Ф-sat-1, an enhancement of the Federated..."
2,Webb,The James Webb Space Telescope (Webb) is the n...,https://www.esa.int/Science_Exploration/Space_...,d2,Webb The James Webb Space Telescope (Webb) is ...
3,Vigil,ESA Vigil will be the first of its kind. By ke...,https://www.esa.int/Space_Safety/Vigil,d3,Vigil ESA Vigil will be the first of its kind....
4,Vega,Vega first launched in 2012 from Europe’s Spac...,https://www.esa.int/Enabling_Support/Space_Tra...,d4,Vega Vega first launched in 2012 from Europe’s...
5,Swarm,ESA’s three-satellite Swarm mission is dedicat...,https://www.esa.int/Applications/Observing_the...,d5,Swarm ESA’s three-satellite Swarm mission is d...
6,Space Rider,Space Rider is an uncrewed robotic laboratory ...,https://www.esa.int/Enabling_Support/Space_Tra...,d6,Space Rider Space Rider is an uncrewed robotic...
7,SMOS,ESA’s Soil Moisture and Ocean Salinity (SMOS) ...,https://www.esa.int/Applications/Observing_the...,d7,SMOS ESA’s Soil Moisture and Ocean Salinity (S...
8,Rosetta,ESA's Rosetta mission was the first to rendezv...,https://www.esa.int/Science_Exploration/Space_...,d8,Rosetta ESA's Rosetta mission was the first to...
9,meteosat third generation,Building on the long-standing partnership betw...,https://www.esa.int/Applications/Observing_the...,d9,meteosat third generation Building on the long...


In [ ]:
df_2 = pd.read_json("./results_spaceflight_no3.json")
ids = []
for i in range(len(df),len(df_2)+len(df)):
    ids.append(f"d{i}")
df_better = pd.DataFrame()
df_better['docno'] = ids
df_better['text'] = f"{df_2['title']} \n {df_2['date']}  \n {df_2['description']} \n {df_2['body'][:-34]}".lower()
# df_better.info()


In [ ]:
texts = []
for i in range(df_2.shape[0]):
    prova = df_2.iloc[i, :]
    texts.append((prova['title'] + " " + prova['description'] + " " + prova['body'][:-34] + " " + prova['date'].strftime("%d-%B-%Y")).lower())
prova['date'].strftime("%d %B %Y")

In [ ]:
df_better['text'] = texts
df_better.info
df_better = pd.concat([df,df_better])

In [3]:
indexer = pt.DFIndexer("./index_3docs", overwrite=True)
index_ref = indexer.index(df["text"], df["docno"])
# index_ref.toString()
index = pt.IndexFactory.of(index_ref)


In [ ]:

#lets see what type index is.
type(index)

In [4]:
print(index.getCollectionStatistics().toString())

Number of documents: 28
Number of terms: 562
Number of postings: 949
Number of fields: 0
Number of tokens: 1134
Field names: []
Positions:   false



In [ ]:
word = 'rocket'
pointer = index.getLexicon()[word]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen=%d" % posting.getDocumentLength())

In [ ]:
# br.search("rocket")
# print(df_better['text'][2])

In [5]:
br = pt.BatchRetrieve(index, wmodel="BM25") #Query with a single word
queries = pd.DataFrame([["q1", "stars"]], columns=["qid", "query"])
br.transform(queries)

,qid,docid,docno,rank,score,query
0,q1,24,d24,0,3.237909,stars
1,q1,14,d14,1,2.616581,stars
2,q1,2,d2,2,2.251476,stars
3,q1,3,d3,3,1.868986,stars


In [ ]:
bm25 = pt.BatchRetrieve(index, num_results = 15, wmodel="BM25")
queries = pd.DataFrame([["q1", "stars solar system"],["q2", "solar system"]], columns=["qid", "query"])
results = bm25.transform(queries)
results

In [ ]:


if __name__ == "__main__":
    createIndex()
    startServer()